In [55]:
# I faced lot of errors with gensim lib, after lot of articles on internet.
# I find a way, it is to degrade some of the packages. so the gesim lib is compatible with them.
# after executing the commented code below and restarting the session
# problem is resolved.

# !pip uninstall -y gensim numpy scipy
# !pip install numpy==1.23.5 scipy==1.10.1 gensim==4.3.1

In [56]:
import pandas as pd
import gensim
from gensim import corpora, models
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt_tab')
nltk.download('stopwords')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
# loading the dataset
df = pd.read_csv('/content/text_docs.csv')
df.head()


,document_id,text
0,1,The stock market has been experiencing volatil...
1,2,"The economy is growing, and businesses are opt..."
2,3,Climate change is a critical issue that needs ...
3,4,Advances in artificial intelligence have revol...
4,5,The rise of electric vehicles is shaping the f...


In [58]:
print("total-rows:", len(df))

print("\nunique documents:", df['text'].nunique())

total-rows: 10

unique documents: 10


In [59]:


def dataCleaning(text):
    text = text.lower()
    # text = re.sub(r'\W+', ' ', text)
    text = text.translate(str.maketrans('', '', string.punctuation))

    # tokenizing
    tokens = word_tokenize(text)

    # removing stopwords- create list from tokens if word not in stopwards
    stop_words = set(stopwords.words('english'))

    return [word for word in tokens if word not in stop_words and len(word) > 2]

# df['tokenized_text'] = df['text'].apply(dataCleaning)
df['tokenized_text'] = [dataCleaning(text) for text in df['text']]

In [60]:
# inspecting data
df[['text', 'tokenized_text']].head()

,text,tokenized_text
0,The stock market has been experiencing volatil...,"[stock, market, experiencing, volatility, due,..."
1,"The economy is growing, and businesses are opt...","[economy, growing, businesses, optimistic, fut..."
2,Climate change is a critical issue that needs ...,"[climate, change, critical, issue, needs, imme..."
3,Advances in artificial intelligence have revol...,"[advances, artificial, intelligence, revolutio..."
4,The rise of electric vehicles is shaping the f...,"[rise, electric, vehicles, shaping, future, au..."


In [61]:
# dictionary
dictionary = corpora.Dictionary(df['tokenized_text'])

print(dictionary)


Dictionary<62 unique tokens: ['due', 'economic', 'experiencing', 'market', 'stock']...>


In [62]:
dataCorpus = [dictionary.doc2bow(tokn) for tokn in df['tokenized_text']]

print(dataCorpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)], [(20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(9, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(28, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)], [(44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)], [(50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)], [(38, 1), (40, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1)]]


In [63]:
# applying LDA with 5 topics
lda_model = models.LdaModel(corpus=dataCorpus, id2word=dictionary, num_topics=5, random_state=51, passes=20)

In [64]:
# getting topics for each document
topics = lda_model.print_topics(num_words=5)

In [65]:
topics

[(0,
  '0.045*"digital" + 0.045*"platforms" + 0.045*"industry" + 0.045*"uncertainty" + 0.045*"entertainment"'),
 (1,
  '0.049*"advances" + 0.049*"media" + 0.049*"interact" + 0.049*"industries" + 0.049*"worldwide"'),
 (2,
  '0.062*"energy" + 0.062*"world" + 0.062*"around" + 0.062*"investing" + 0.062*"projects"'),
 (3,
  '0.035*"integrated" + 0.035*"automobile" + 0.035*"ongoing" + 0.035*"rise" + 0.035*"cybersecurity"'),
 (4,
  '0.051*"future" + 0.051*"healthcare" + 0.051*"introduction" + 0.051*"treatments" + 0.051*"technologies"')]

In [66]:
topics_df = pd.DataFrame(topics, columns=['topic', 'words'])
topics_df

,topic,words
0,0,"0.045*""digital"" + 0.045*""platforms"" + 0.045*""i..."
1,1,"0.049*""advances"" + 0.049*""media"" + 0.049*""inte..."
2,2,"0.062*""energy"" + 0.062*""world"" + 0.062*""around..."
3,3,"0.035*""integrated"" + 0.035*""automobile"" + 0.03..."
4,4,"0.051*""future"" + 0.051*""healthcare"" + 0.051*""i..."
